# HW2 Text Clustering

In [1]:
!pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import csv
from sentence_transformers import SentenceTransformer, util
# We donwload the Quora Duplicate Questions Dataset (https://www.quora.com/q/quoradata/First-Quora-Dataset-Release-Question-Pairs)
# and find similar question in it
url = "http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv"
dataset_path = "quora_duplicate_questions.tsv"
max_corpus_size = 5000  # We limit our corpus to only the first 50k questions


# Check if the dataset exists. If not, download and extract
# Download dataset if needed
if not os.path.exists(dataset_path):
    print("Download dataset")
    util.http_get(url, dataset_path)

# Get all unique sentences from the file
corpus_sentences = set()
with open(dataset_path, encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_MINIMAL)
    for row in reader:
        corpus_sentences.add(row['question1'])
        corpus_sentences.add(row['question2'])
        if len(corpus_sentences) >= max_corpus_size:
            break

corpus_sentences = list(corpus_sentences)
print("Encode the corpus. This might take a while")

embedder = SentenceTransformer('all-MiniLM-L6-v2')
corpus_embeddings = embedder.encode(corpus_sentences)

Encode the corpus. This might take a while


## 2-3: Embedding dimension reduction

In [3]:
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer, LoggingHandler, util, evaluation, models, InputExample
import os
import gzip
import csv
import random
import numpy as np
import torch

In [4]:
new_dimension = 128
#Compute PCA on the train embeddings matrix
pca = PCA(n_components=new_dimension)
reduced_emb = pca.fit_transform(corpus_embeddings)


In [5]:
reduced_emb.shape[1]

128

In [10]:
%%time
from sklearn.cluster import KMeans
# Perform kmean clustering
num_clusters = 5
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(reduced_emb)
cluster_assignment = clustering_model.labels_

CPU times: user 1.54 s, sys: 1.07 s, total: 2.61 s
Wall time: 1.49 s


In [11]:
len(set(cluster_assignment))

5

In [14]:
%%time
from sklearn.cluster import AgglomerativeClustering
clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(reduced_emb)
cluster_assignment = clustering_model.labels_

CPU times: user 2.09 s, sys: 15.8 ms, total: 2.1 s
Wall time: 2.09 s


In [15]:
len(set(cluster_assignment))

5